In [1]:
# https://docs.microsoft.com/en-us/azure/machine-learning/how-to-access-data#python-sdk

In [2]:
from azureml.core import Workspace
ws = Workspace.from_config()

### Create Datastore for sample images (from public account)

In [3]:
from azureml.core.datastore import Datastore

In [4]:
batchscore_blob = Datastore.register_azure_blob_container(ws, 
                      datastore_name="images_datastore", 
                      container_name="sampledata", 
                      account_name="pipelinedata", 
                      overwrite=True)
def_data_store = ws.get_default_datastore()

### Create Dataset Objects

In [5]:
from azureml.core.dataset import Dataset
from azureml.pipeline.core import PipelineData

In [6]:
input_images = Dataset.File.from_files((batchscore_blob, "batchscoring/images/"))
label_ds = Dataset.File.from_files((batchscore_blob, "batchscoring/labels/"))
output_dir = PipelineData(name="scores", 
                          datastore=def_data_store, 
                          output_path_on_compute="batchscoring/results")

In [7]:
def_data_store

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-2dd703c0-9f8b-4e31-bb4d-9ff3f50aa329",
  "account_name": "amlworkbook2803154736",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [8]:
input_images = input_images.register(workspace = ws, name = "input_images")
label_ds = label_ds.register(workspace = ws, name = "label_ds")

### Download and register the model

In [9]:
import os
import tarfile
import urllib.request

if not os.path.isdir("models"):
    os.mkdir("models")

if not os.path.exists("./models/inception_v3.ckpt"):
    response = urllib.request.urlretrieve("http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz", "model.tar.gz")
    tar = tarfile.open("model.tar.gz", "r:gz")
    tar.extractall("models")

In [10]:
from azureml.core.model import Model
 
model = Model.register(model_path="models/inception_v3.ckpt",
                       model_name="inception",
                       tags={"pretrained": "inception"},
                       description="Imagenet trained tensorflow inception",
                       workspace=ws)

Registering model inception


### Create and Attach the Remote Compute Target

In [11]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.exceptions import ComputeTargetException
compute_name = "gpu-cluster-js01"

# checks to see if compute target already exists in workspace, else create it
try:
    compute_target = ComputeTarget(workspace=ws, name=compute_name)
except ComputeTargetException:
    config = AmlCompute.provisioning_configuration(vm_size="STANDARD_NC6",
                                                   vm_priority="lowpriority", 
                                                   min_nodes=0, 
                                                   max_nodes=1)

    compute_target = ComputeTarget.create(workspace=ws, name=compute_name, provisioning_configuration=config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


### Write a scoring script

In [12]:
%%writefile batch_scoring.py

import os
import argparse
import datetime
import time
import tensorflow as tf
from math import ceil
import numpy as np
import shutil
from tensorflow.contrib.slim.python.slim.nets import inception_v3

from azureml.core import Run
from azureml.core.model import Model
from azureml.core.dataset import Dataset

slim = tf.contrib.slim

image_size = 299
num_channel = 3


def get_class_label_dict(labels_dir):
    label = []
    labels_path = os.path.join(labels_dir, 'labels.txt')
    proto_as_ascii_lines = tf.gfile.GFile(labels_path).readlines()
    for l in proto_as_ascii_lines:
        label.append(l.rstrip())
    return label


def init():
    global g_tf_sess, probabilities, label_dict, input_images

    parser = argparse.ArgumentParser(description="Start a tensorflow model serving")
    parser.add_argument('--model_name', dest="model_name", required=True)
    parser.add_argument('--labels_dir', dest="labels_dir", required=True)
    args, _ = parser.parse_known_args()

    label_dict = get_class_label_dict(args.labels_dir)
    classes_num = len(label_dict)

    with slim.arg_scope(inception_v3.inception_v3_arg_scope()):
        input_images = tf.placeholder(tf.float32, [1, image_size, image_size, num_channel])
        logits, _ = inception_v3.inception_v3(input_images,
                                              num_classes=classes_num,
                                              is_training=False)
        probabilities = tf.argmax(logits, 1)

    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    g_tf_sess = tf.Session(config=config)
    g_tf_sess.run(tf.global_variables_initializer())
    g_tf_sess.run(tf.local_variables_initializer())

    model_path = Model.get_model_path(args.model_name)
    saver = tf.train.Saver()
    saver.restore(g_tf_sess, model_path)


def file_to_tensor(file_path):
    image_string = tf.read_file(file_path)
    image = tf.image.decode_image(image_string, channels=3)

    image.set_shape([None, None, None])
    image = tf.image.resize_images(image, [image_size, image_size])
    image = tf.divide(tf.subtract(image, [0]), [255])
    image.set_shape([image_size, image_size, num_channel])
    return image


def run(mini_batch):
    result_list = []
    for file_path in mini_batch:
        test_image = file_to_tensor(file_path)
        out = g_tf_sess.run(test_image)
        result = g_tf_sess.run(probabilities, feed_dict={input_images: [out]})
        result_list.append(os.path.basename(file_path) + ": " + label_dict[result[0]])
    return result_list

Overwriting batch_scoring.py


### Build the pipeline

In [13]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_GPU_IMAGE

cd = CondaDependencies.create(pip_packages=["tensorflow-gpu==1.15.2",
                                            "azureml-core", "azureml-dataprep[fuse]"])
env = Environment(name="parallelenv")
env.python.conda_dependencies = cd
env.docker.base_image = DEFAULT_GPU_IMAGE

In [14]:
from azureml.pipeline.steps import ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    environment=env,
    entry_script="batch_scoring.py",
    source_directory=".",
    output_action="append_row",
    mini_batch_size="20",
    error_threshold=1,
    compute_target=compute_target,
    process_count_per_node=2,
    node_count=1
)

### Create the Pipeline Step

In [15]:
from azureml.pipeline.steps import ParallelRunStep
from datetime import datetime

parallel_step_name = "batchscoring-" + datetime.now().strftime("%Y%m%d%H%M")

label_config = label_ds.as_named_input("labels_input")

batch_score_step = ParallelRunStep(
    name=parallel_step_name,
    inputs=[input_images.as_named_input("input_images")],
    output=output_dir,
    arguments=["--model_name", "inception",
               "--labels_dir", label_config],
    side_inputs=[label_config],
    parallel_run_config=parallel_run_config,
    allow_reuse=False
)

### Submit the Pipeline

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace=ws, steps=[batch_score_step])
pipeline_run = Experiment(ws, 'batch_scoring').submit(pipeline)
pipeline_run.wait_for_completion(show_output=True)

Created step batchscoring-202010211814 [dfc1df02][ee2484bc-8dd1-4175-92e2-6aedf9a55d3c], (This step will run and generate new outputs)
Using data reference input_images_0 for StepId [1dfdec29][d9e70b7f-3a5b-4292-b7bc-7fb2b00e0139], (Consumers of this data are eligible to reuse prior runs.)Using data reference labels_input_0 for StepId [27256dad][ffd377a9-d8f7-4adc-a2c6-ac66eea45dc0], (Consumers of this data are eligible to reuse prior runs.)

Submitted PipelineRun 54ecf2a5-b46f-4b50-bdd1-de706cddaf4c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/54ecf2a5-b46f-4b50-bdd1-de706cddaf4c?wsid=/subscriptions/b060ea58-a590-43cc-86ea-8ee676be2a76/resourcegroups/aml-workbook/workspaces/aml-workbook
PipelineRunId: 54ecf2a5-b46f-4b50-bdd1-de706cddaf4c
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/batch_scoring/runs/54ecf2a5-b46f-4b50-bdd1-de706cddaf4c?wsid=/subscriptions/b060ea58-a590-43cc-86ea-8ee676be2a76/resourcegroups/aml-w


sqlite-3.23.1        | 1.5 MB    |            |   0% 
sqlite-3.23.1        | 1.5 MB    | ########## | 100% 

python-3.6.2         | 27.0 MB   |            |   0% 
python-3.6.2         | 27.0 MB   | #5         |  16% 
python-3.6.2         | 27.0 MB   | ####2      |  43% 
python-3.6.2         | 27.0 MB   | ######7    |  68% 
python-3.6.2         | 27.0 MB   | #########4 |  94% 
python-3.6.2         | 27.0 MB   | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Ran pip subprocess with arguments:
['/azureml-envs/azureml_b0827357ec606a0be8df05ade41cd431/bin/python', '-m', 'pip', 'install', '-U', '-r', '/azureml-environment-setup/condaenv.ldrqo5wb.requirements.txt']
Pip subprocess output:
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7540 sha256=3d948711f79f3ce15894693dc27744bc5b473cabdba8a0a97d8f7973e328b295
  Stored in directory: /root/.cache/pip/wheels/19/a7/b9/0740c7a3a7

Removing intermediate container bacfd4703541
 ---> 7459fa485963
Step 9/15 : ENV PATH /azureml-envs/azureml_b0827357ec606a0be8df05ade41cd431/bin:$PATH
 ---> Running in 3065f9588586
Removing intermediate container 3065f9588586
 ---> 6db3bb68d752
Step 10/15 : ENV AZUREML_CONDA_ENVIRONMENT_PATH /azureml-envs/azureml_b0827357ec606a0be8df05ade41cd431
 ---> Running in 49b53c9cdb7a
Removing intermediate container 49b53c9cdb7a
 ---> 5d052b5075e0
Step 11/15 : ENV LD_LIBRARY_PATH /azureml-envs/azureml_b0827357ec606a0be8df05ade41cd431/lib:$LD_LIBRARY_PATH
 ---> Running in 4ca882a8ffa2
Removing intermediate container 4ca882a8ffa2
 ---> d582a66534f5
Step 12/15 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> 86fb107b618f
Step 13/15 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit  /azureml-environment-setup/spark_cache.py'; fi
 ---> Running in a1e80edc6d1b
Removing intermediate container a1e80edc6d1b

### Download and review output

In [ ]:
import pandas as pd

batch_run = next(pipeline_run.get_children())
batch_output = batch_run.get_output_data("scores")
batch_output.download(local_path="inception_results")

for root, dirs, files in os.walk("inception_results"):
    for file in files:
        if file.endswith("parallel_run_step.txt"):
            result_file = os.path.join(root, file)

df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["Filename", "Prediction"]
print("Prediction has ", df.shape[0], " rows")
df.head(10)